# Minibatch FFNN Implementation Evaluation
### Group Members 
1. 13516044 - Annisa Sekar Ayuningtyas
2. 13516078 - Muhammad Alif Arifin
3. 13516152 - Deborah Aprilia Josephine

## Import Library and Dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from MiniBatch import MiniBatch

df = pd.read_csv('dataset.csv')
df

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


In [2]:
def rationalize(list_df) :
    max_df = list_df.max()
    min_df = list_df.min()
    result = list(map(lambda x: (x-min_df)/(max_df-min_df)*1, list_df))
    return result

df['temperature'] = rationalize(df['temperature'])
df['humidity'] = rationalize(df['humidity'])
df

,outlook,temperature,humidity,windy,play
0,sunny,1.000000,0.645161,False,no
1,sunny,0.761905,0.806452,True,no
2,overcast,0.904762,0.677419,False,yes
3,rainy,0.285714,1.000000,False,yes
4,rainy,0.190476,0.483871,False,yes
5,rainy,0.047619,0.161290,True,no
6,overcast,0.000000,0.000000,True,yes
7,sunny,0.380952,0.967742,False,no
8,sunny,0.238095,0.161290,False,yes
9,rainy,0.523810,0.483871,False,yes


In [3]:
df['outlook'] = pd.Categorical(df['outlook'])
dfOutlook = pd.get_dummies(df['outlook'], prefix = 'outlook')
dfOutlook

,outlook_overcast,outlook_rainy,outlook_sunny
0,0,0,1
1,0,0,1
2,1,0,0
3,0,1,0
4,0,1,0
5,0,1,0
6,1,0,0
7,0,0,1
8,0,0,1
9,0,1,0


In [4]:
df['windy'] = pd.Categorical(df['windy'])
dfWindy = pd.get_dummies(df['windy'], prefix = 'windy')
dfWindy

# df['windy'] = df['windy'].replace(True, 1)
# df['windy'] = df['windy'].replace(False, -1)
# df

,windy_False,windy_True
0,1,0
1,0,1
2,1,0
3,1,0
4,1,0
5,0,1
6,0,1
7,1,0
8,1,0
9,1,0


In [5]:
# df['play'] = df['play'].replace('yes', 1)
# df['play'] = df['play'].replace('no', -1)

df['play'] = df['play'].astype('category')
df['play'] = df['play'].cat.codes

df

,outlook,temperature,humidity,windy,play
0,sunny,1.000000,0.645161,False,0
1,sunny,0.761905,0.806452,True,0
2,overcast,0.904762,0.677419,False,1
3,rainy,0.285714,1.000000,False,1
4,rainy,0.190476,0.483871,False,1
5,rainy,0.047619,0.161290,True,0
6,overcast,0.000000,0.000000,True,1
7,sunny,0.380952,0.967742,False,0
8,sunny,0.238095,0.161290,False,1
9,rainy,0.523810,0.483871,False,1


In [6]:
df = pd.concat([df, dfOutlook, dfWindy], axis=1)
df = df.drop(columns=['outlook', 'windy'], axis=1)

# df = pd.concat([df, dfOutlook], axis=1)
# df = df.drop(columns=['outlook'], axis=1)

df

,temperature,humidity,play,outlook_overcast,outlook_rainy,outlook_sunny,windy_False,windy_True
0,1.000000,0.645161,0,0,0,1,1,0
1,0.761905,0.806452,0,0,0,1,0,1
2,0.904762,0.677419,1,1,0,0,1,0
3,0.285714,1.000000,1,0,1,0,1,0
4,0.190476,0.483871,1,0,1,0,1,0
5,0.047619,0.161290,0,0,1,0,0,1
6,0.000000,0.000000,1,1,0,0,0,1
7,0.380952,0.967742,0,0,0,1,1,0
8,0.238095,0.161290,1,0,0,1,1,0
9,0.523810,0.483871,1,0,1,0,1,0


In [7]:
features = df.drop(columns=['play'], axis=1)
targets = df['play']

print (features.head)
print ()
print (targets.head)

<bound method NDFrame.head of     temperature  humidity  outlook_overcast  outlook_rainy  outlook_sunny  \
0      1.000000  0.645161                 0              0              1   
1      0.761905  0.806452                 0              0              1   
2      0.904762  0.677419                 1              0              0   
3      0.285714  1.000000                 0              1              0   
4      0.190476  0.483871                 0              1              0   
5      0.047619  0.161290                 0              1              0   
6      0.000000  0.000000                 1              0              0   
7      0.380952  0.967742                 0              0              1   
8      0.238095  0.161290                 0              0              1   
9      0.523810  0.483871                 0              1              0   
10     0.523810  0.161290                 0              0              1   
11     0.380952  0.806452                 1   

## Training and Testing

In [8]:
features_train, features_test, targets_train, targets_test = train_test_split(features, targets, test_size=0.5, stratify=targets, random_state=None)
print (features_train)
print ()
print (targets_train)
print ()

print (features_test)
print ()
print (targets_test)
print ()

    temperature  humidity  outlook_overcast  outlook_rainy  outlook_sunny  \
10     0.523810  0.161290                 0              0              1   
8      0.238095  0.161290                 0              0              1   
5      0.047619  0.161290                 0              1              0   
3      0.285714  1.000000                 0              1              0   
1      0.761905  0.806452                 0              0              1   
13     0.333333  0.838710                 0              1              0   
11     0.380952  0.806452                 1              0              0   

    windy_False  windy_True  
10            0           1  
8             1           0  
5             0           1  
3             1           0  
1             0           1  
13            0           1  
11            0           1  

10    1
8     1
5     0
3     1
1     0
13    0
11    1
Name: play, dtype: int8

    temperature  humidity  outlook_overcast  outlook_rainy  o

In [9]:
# nb nodes, hidden layer, batch size, learning rate, momentum, epoch
model = MiniBatch(10, 9, 2, 0.25, 0.005, 25)
model.fit(features_train, targets_train)
print ('Predict')
predict = model.predict(features_test)

print (predict)
# print (len(predict))
# print (len(targets_test))

print ('Accuracy\t', accuracy_score(targets_test, predict))

Predict
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Accuracy	 0.2857142857142857
